In [1]:
import fitz  # PyMuPDF
import nltk
from nltk.tokenize import word_tokenize


import pdfplumber
import pandas as pd
import json


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rikardforlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rikardforlin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Tokenize flat-sequence, but with delimiters

In [3]:


def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def tokenize_text(text):
    """Tokenize the extracted text."""
    return word_tokenize(text)


In [6]:

pdf_path = "/Users/rikardforlin/Downloads/strama_behandling.pdf"  # Path to PDF
text = extract_text_from_pdf(pdf_path)
tokens = tokenize_text(text)

# Print first 100 tokens as a preview
print(tokens[:100])


['gäller', 'från', '2023-10-24', 'Behandlingsrekommendationer', 'för', 'vanliga', 'infektioner', 'i', 'öppenvård', 'Tecken', 'på', 'allvarlig', 'infektion', 'hos', 'vuxna', 'och', 'barn', 'Akut', 'mediaotit', 'Rinosinuit', 'Faryngotonsillit', 'Akut', 'bronkit', 'och', 'pneumoni', 'Akut', 'exacerbation', 'av', 'KOL', 'Urinvägsinfektioner', 'Hud-', 'och', 'mjukdelsinfektioner', 'Sexuellt', 'överförbara', 'bakteriella', 'infektioner', 'Detta', 'är', 'en', 'digital', 'version', 'av', 'Behandlingsrekommendationer', 'för', 'vanliga', 'infektioner', 'i', 'öppenvård', '.', 'Den', 'senaste', 'uppdateringen', 'gjordes', '2023-10-24', '.', 'Om', 'du', 'har', 'en', 'pappersbroschyr', 'kan', 'den', 'skilja', 'sig', 'något', 'från', 'denna', 'digitala', 'version', '.', '2', '·', 'Behandlingsrekommendationer', 'för', 'vanliga', 'infektioner', 'i', 'öppenvård', '2023', 'Förkortningar', 'ABU', 'Asymtomatisk', 'bakteriuri', 'AOM', 'Akut', 'mediaotit', 'CRP', 'C-reaktivt', 'protein', 'CRT', 'Capillary', 

In [23]:
for i, t in enumerate(tokens):
    if t == 'antibiotikabehandling':
        if tokens[i+1] == 'vid':
            if tokens[i+2] == 'febril':
                if tokens[i+3] == 'UVI':
                    if tokens[i+4] == 'hos':
                        if tokens[i+5] == 'män':
                            break

# Tokenize to JSON

In [6]:

pdf_path = "/Users/rikardforlin/Downloads/strama_behandling.pdf"  # Path to PDF

def extract_tables_from_pdf(pdf_path):
    """Extract tables from a PDF and return structured data."""
    structured_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                headers = table[0]  # Assuming first row is header
                for row in table[1:]:
                    structured_data.append(dict(zip(headers, row)))
    return structured_data

def save_json(data, output_path):
    """Save extracted table data as JSON."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)




In [8]:

structured_data = extract_tables_from_pdf(pdf_path)
output_json_path = "/mnt/data/structured_data.json"
#save_json(structured_data, output_json_path)
print(f"Structured data saved to {output_json_path}")

Structured data saved to /mnt/data/structured_data.json


# Tokenize flat-sequence, but with delimiters

In [10]:

pdf_path = "/Users/rikardforlin/Downloads/strama_behandling.pdf"  # Path to PDF

def extract_tables_from_pdf(pdf_path):
    """Extract tables from a PDF and return structured data."""
    structured_data = []
    delimited_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                headers = table[0]  # Assuming first row is header
                for row in table[1:]:
                    structured_data.append(dict(zip(headers, row)))
                    delimited_tables.append(" | ".join(row))
    return structured_data, delimited_tables

def save_json(data, output_path):
    """Save extracted table data as JSON."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def save_delimited(data, output_path):
    """Save extracted table data in a delimited format."""
    with open(output_path, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(line + "\n")


In [11]:
structured_data, delimited_tables = extract_tables_from_pdf(pdf_path)
output_json_path = "/mnt/data/structured_data.json"
output_delimited_path = "/mnt/data/structured_data.txt"
#save_json(structured_data, output_json_path)
#save_delimited(delimited_tables, output_delimited_path)
print(f"Structured data saved to {output_json_path}")
print(f"Delimited table data saved to {output_delimited_path}")


Structured data saved to /mnt/data/structured_data.json
Delimited table data saved to /mnt/data/structured_data.txt


In [ ]:
# https://github.com/cactiML/clinitokenizer
# https://huggingface.co/AdaptLLM/medicine-LLM

In [21]:


def clean_text(text):
    """Remove line breaks and extra spaces from text."""
    if text:
        return re.sub(r'\s+', ' ', text).strip()
    return "N/A"

def extract_tables_from_pdf(pdf_path):
    """Extract tables from a PDF and return structured data."""
    structured_data = []
    delimited_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                headers = [clean_text(header) for header in table[0]]  # Clean headers
                for row in table[1:]:
                    cleaned_row = [clean_text(cell) for cell in row]
                    structured_data.append(dict(zip(headers, cleaned_row)))
                    delimited_tables.append(" | ".join(cleaned_row))
    return structured_data, delimited_tables

def save_json(data, output_path):
    """Save extracted table data as JSON."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def save_delimited(data, output_path):
    """Save extracted table data in a delimited format."""
    with open(output_path, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(line + "\n")



In [18]:
import re

In [19]:


structured_data, delimited_tables = extract_tables_from_pdf(pdf_path)
output_json_path = "/mnt/data/structured_data.json"
output_delimited_path = "/mnt/data/structured_data.txt"
#save_json(structured_data, output_json_path)
#save_delimited(delimited_tables, output_delimited_path)
print(f"Structured data saved to {output_json_path}")
print(f"Delimited table data saved to {output_delimited_path}")


Structured data saved to /mnt/data/structured_data.json
Delimited table data saved to /mnt/data/structured_data.txt


In [33]:
structured_data

[{'Grönt ljus Låg risk': 'Normalt/alert',
  'Gult ljus Medelhög risk': 'Anamnes på förändrat beteende',
  'Rött ljus Hög risk': 'Objektivt för- ändrat beteende/ ej alert'},
 {'Grönt ljus Låg risk': '12–20 andetag/ min',
  'Gult ljus Medelhög risk': '21–24 andetag/min',
  'Rött ljus Hög risk': '≥ 25 andetag/min'},
 {'Grönt ljus Låg risk': '≥ 96 % (a)',
  'Gult ljus Medelhög risk': '92–95 % (a)',
  'Rött ljus Hög risk': '< 92 % (< 88 % vid kronisk obstruktiv lung- sjukdom)'},
 {'Grönt ljus Låg risk': '> 100 mmHg',
  'Gult ljus Medelhög risk': '91–100 mmHg',
  'Rött ljus Hög risk': '≤ 90 mmHg eller > 40 mmHg lägre än patientens normala tryck'},
 {'Grönt ljus Låg risk': '≤ 90 /min',
  'Gult ljus Medelhög risk': '91–130/min eller nytillkommen arytmi',
  'Rött ljus Hög risk': '> 130/min'},
 {'Grönt ljus Låg risk': '≥ 36 °C',
  'Gult ljus Medelhög risk': '< 36 °C',
  'Rött ljus Hög risk': 'N/A'},
 {'Grönt ljus Låg risk': 'N/A',
  'Gult ljus Medelhög risk': 'Ej kissat på 12–17 tim',
  'Rött lj